# Notebook details

In [ ]:
def setup_notebook(fix_python_path=True, reduce_margins=True, plot_inline=True):
    if reduce_margins:
        # Reduce side margins of the notebook
        from IPython.core.display import display, HTML
        display(HTML("<style>.container { width:100% !important; }</style>"))

    if fix_python_path:
        # add egosocial to the python path
        import os, sys
        sys.path.extend([os.path.dirname(os.path.abspath('.'))])

    if plot_inline:
        # Plots inside cells
        %matplotlib inline
    
    global __file__
    __file__ = 'Notebook'

setup_notebook()

# Imports and Constants Definition

In [ ]:
# !/usr/bin/env python
# -*- coding: utf-8 -*-

import argparse
import itertools
import json
import logging
import os

from google_drive_downloader import GoogleDriveDownloader as gdd
import numpy as np
import pandas as pd

import keras
from keras import backend as K
from keras.models import Model
from keras.layers import Flatten
from keras.applications import imagenet_utils
from keras.applications.resnet50 import ResNet50
import keras.preprocessing.image
from keras.utils import to_categorical

import egosocial
import egosocial.config
from egosocial.core.attributes import AttributeSelector
from egosocial.utils.filesystem import create_directory
from egosocial.utils.filesystem import check_directory
from egosocial.utils.keras.backend import limit_gpu_allocation_tensorflow
from egosocial.utils.keras.layers import LRN
from egosocial.utils.logging import setup_logging

# Limit GPU memory allocation with Tensorflow

In [ ]:
limit_memory = False
if limit_memory and K.backend() == 'tensorflow':
    memory_ratio = 0.6
    limit_gpu_allocation_tensorflow(memory_ratio)

# Fake main

In [ ]:
def main(*fake_args):
    entry_msg = 'Extract features for egosocial photo-streams.'
    parser = argparse.ArgumentParser(description=entry_msg)

    parser.add_argument('--dataset_path', required=True,
                        help='Path to file containing the input data and labels information merged.')

    parser.add_argument('--features_dir', required=True,
                        help='Directory where the extracted features are stored.') 
    
    parser.add_argument('--batch_size', required=False, type=int,
                        default=32,
                        help='Batch size.')    
    
    if not os.path.isdir(egosocial.config.TMP_DIR):
        os.mkdir(egosocial.config.TMP_DIR)

    setup_logging(egosocial.config.LOGGING_CONFIG,
                  log_dir=egosocial.config.LOGS_DIR)
    
    # TODO: implement correctly
    args = parser.parse_args(*fake_args)

    return args

In [ ]:
BASE_DIR = os.path.join(egosocial.config.TMP_DIR, 'egocentric', 'datasets')

args = [
    "--dataset_path", os.path.join(BASE_DIR, 'merged_dataset.json'),
    "--features_dir", os.path.join(BASE_DIR, 'extracted_features'),
]

conf = main(args)

create_directory(conf.features_dir)

# Helper functions

In [ ]:
def get_activity_model(layer=None, img_width=224, img_height=224):
    base_model = ResNet50(weights=None, include_top=False, input_shape=(img_width, img_height, 3))
    
    if layer:
        layer_output = base_model.get_layer(layer).output
    else:
        layer_output = Flatten(name='flatten')(base_model.output)
        
    model = Model(inputs=base_model.input, outputs=[layer_output])

    weights = os.path.join(egosocial.config.MODELS_CACHE_WEIGHTS, 'ego_activity_weights_resNet50.hdf5')
    model.load_weights(weights, by_name=True)
    
    return model

def get_models(query, layer=None):
    _log = logging.getLogger(os.path.basename(__file__))
    
    selector = AttributeSelector(egosocial.config.MODEL_KEYS)
    attributes = selector.filter(query)
    
    directory = egosocial.config.MODELS_CACHE_FULL
    if attributes and not os.path.isdir(directory):
        create_directory(directory, 'Models cache')

    models = {}
    for attr in attributes:
        model_info = egosocial.config.MODELS[attr]        
        
        # absolute path to the file
        model_file = os.path.join(directory, model_info['file'])        
        gdd.download_file_from_google_drive(
            file_id=model_info['file_id'],
            dest_path=model_file,
        )
                
        _log.debug('Loading model from {}'.format(model_file))        
        features_model = keras.models.load_model(
            model_file, custom_objects={'LRN': LRN}
        )
        
        if layer:
            layer_output = features_model.get_layer(layer).output
            features_model = Model(
                inputs=features_model.inputs,
                outputs=[layer_output],
            )

        models[attr] = features_model
    
    return models

# Main class

In [ ]:
def load_dataset_defition(dataset_path):
    with open(dataset_path, 'r') as json_file:
        dataset_def = json.load(json_file)

    # flatten the segments structure
    samples = pd.DataFrame(list(itertools.chain(*dataset_def)))
    return samples

class FeatureExtractor:
    
    def __init__(
        self,
        dataset_path,
        extract_features_cbk,
        data_type,
        features_path=None,
        preprocessing_function=None,
        target_size=None,
        batch_size=32,
    ):
        assert data_type in ('body', 'face', 'context', 'camera_user_data')
        assert batch_size >= 1
        
        self._dataset_path = dataset_path
        self._extract_features_cbk = extract_features_cbk        
        self._data_type = data_type
        self._features_path = features_path
        
        self._preprocessing_function = preprocessing_function
        self._target_size = target_size
        self._batch_size = batch_size
        
        # set up logging
        self._log = logging.getLogger(self.__class__.__name__)
    
    def run(self):
        self._log.debug('Loading dataset definition: {}'.format(self._dataset_path))        
        samples = load_dataset_defition(self._dataset_path)
        
        n_samples = len(samples)        
        n_batches = int(np.ceil(n_samples / self._batch_size))
        
        features = []
        for batch_i in range(n_batches):
            self._log.debug('Processing batch {} / {}'.format(batch_i, n_batches))
            
            batch_index = range(batch_i * self._batch_size, min((batch_i+1) * self._batch_size, n_samples))
            
            if self._data_type == 'camera_user_data':
                data_batch = samples.loc[batch_index, ['camera_user_age', 'camera_user_gender']]
            else:
                data_batch = self._load_images_batch(samples, batch_index)
            
            self._log.debug('Extracting features...')            
            features_batch = self._extract_features_cbk(data_batch)
            features.append(features_batch)
            
        features_array = np.concatenate(features)

        if self._features_path:
            self._save_features(self._features_path, features_array)
        
        return features_array
    
    def _load_images_batch(self, samples, batch_idxs):
        images_batch = np.zeros((len(batch_idxs),) + self._target_size + (3,))
        
        for local_idx, global_idx in enumerate(batch_idxs):
            self._log.debug('Loading item {}'.format(global_idx))            
            sample_dict = samples.loc[global_idx]
            
            if self._data_type == 'body':
                image_path = sample_dict['body_image_path']
            elif self._data_type == 'face':
                image_path = sample_dict['face_image_path']
            elif self._data_type == 'context':
                image_path = sample_dict['global_image_path']
            
            images_batch[local_idx] = self._load_image_array(image_path)

        return images_batch
        
    def _save_features(self, output_path, features_array):
        self._log.debug('Saving features to: {}'.format(output_path))        
        np.save(output_path, features_array)
    
    def _load_image_array(self, image_path):
        self._log.debug('Loading image: {}'.format(image_path))
        
        img = keras.preprocessing.image.load_img(
            image_path, 
            target_size=self._target_size
        )

        x = keras.preprocessing.image.img_to_array(img)
        
        if self._preprocessing_function:
            x = self._preprocessing_function(x)
        
        return x

In [ ]:
#batch_size = conf.batch_size
batch_size = 64

In [ ]:
for data_type in ('body', 'face'):
    models = get_models(data_type, layer='relu7')
    
    for attribute, model in models.items():
        model.compile(optimizer='adam', loss='categorical_crossentropy')
        extract_features_cbk = lambda x : model.predict(x, batch_size=batch_size)

        features_path = os.path.join(conf.features_dir, attribute + '.npy')

        feature_extractor = FeatureExtractor(
                dataset_path=conf.dataset_path,
                extract_features_cbk=model.predict,
                data_type=data_type,
                features_path=features_path,
                preprocessing_function=imagenet_utils.preprocess_input,
                target_size=(227, 227),
                batch_size=batch_size,
        )
        
        _ = feature_extractor.run()

In [ ]:
attribute = 'activity'
data_type = 'context'
features_path = os.path.join(conf.features_dir, attribute + '.npy')

model = get_activity_model()
model.compile(optimizer='adam', loss='categorical_crossentropy')

rescale_0_1 = lambda x : x / 255.0
extract_features_cbk = lambda x : model.predict(x, batch_size=batch_size)

feature_extractor = FeatureExtractor(
        dataset_path=conf.dataset_path,
        extract_features_cbk=extract_features_cbk,
        data_type=data_type,
        features_path=features_path,
        preprocessing_function=rescale_0_1,
        target_size=(224, 224),
        batch_size=batch_size,
)

_ = feature_extractor.run()

In [ ]:
class CategoricalEncoderHelper:
    def __init__(self, categories, attribute_idx):
        self.encoding_map = {cls:i for i, cls in enumerate(categories)}        
        self.encode = np.vectorize(lambda cls : self.encoding_map[cls])        
        self.attribute_idx = attribute_idx

    def __call__(self, batch_x):
        attribute_batch = batch_x[self.attribute_idx]
        return to_categorical(self.encode(attribute_batch), num_classes=len(self.encoding_map))

In [ ]:
camera_user_attributes = [
    ('camera_user_gender', ('male', 'female')),
    ('camera_user_age', ('infant', 'child', 'young', 'middleAge', 'senior', 'unknown')),
]

data_type = 'camera_user_data'

for attribute, categories in camera_user_attributes:

    features_path = os.path.join(conf.features_dir, attribute + '.npy')
    encoder_cbk = CategoricalEncoderHelper(categories, attribute)

    feature_extractor = FeatureExtractor(
        dataset_path=conf.dataset_path,
        extract_features_cbk=encoder_cbk,
        data_type=data_type,
        features_path=features_path,
        batch_size=batch_size,
    )

    _ = feature_extractor.run()

In [ ]:
os.listdir(conf.features_dir)